In [1]:
import dense_correspondence_manipulation.utils.utils as utils
utils.add_dense_correspondence_to_python_path()
import dense_correspondence
reload(dense_correspondence)
from dense_correspondence.training.training import *
import sys
import logging
import json
import time

#utils.set_default_cuda_visible_devices()
# utils.set_cuda_visible_devices([0]) # use this to manually set CUDA_VISIBLE_DEVICES

import dense_correspondence.dataset.spartan_dataset_masked
reload(dense_correspondence.dataset.spartan_dataset_masked)
import dense_correspondence.training.training
reload(dense_correspondence.training.training)
from dense_correspondence.training.training import DenseCorrespondenceTraining
from dense_correspondence.dataset.spartan_dataset_masked import SpartanDataset, OPDataSelector
logging.basicConfig(level=logging.INFO)

import dense_correspondence.evaluation.evaluation
reload(dense_correspondence.evaluation.evaluation)
from dense_correspondence.evaluation.evaluation import DenseCorrespondenceEvaluation

# Multi Object Pursuit
utils.add_object_pursuit_to_python_path()
import object_pursuit
reload(object_pursuit)
from dense_correspondence.training.op_training import OPDenseCorrespondenceTraining

from dense_correspondence.network.op_dense_correspondence_network import \
    OPMultiDenseCorrespondenceNetwork, OPMultiDenseCorrespondenceNetworkV2

cuda = torch.cuda.is_available()
DEVICE = "cuda:1" if cuda else "cpu"
if cuda:
    print("CUDA GPU!")
else:
    print("CPU!")
    
PRETRAINED_ROOT = "/home/ashek/code/data/pdc/trained_models/tutorials"
!CUDA_LAUNCH_BLOCKING=1

CUDA GPU!


## Load Finetuning Dataset

In [2]:
verbose = False
config_filename = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 'dense_correspondence', 
                               'dataset', 'composite', 'object_pursuit.yaml')
data_config = utils.getDictFromYamlFilename(config_filename)
data_config['logs_root_path'] = '/home/ashek/code/data/pdc/logs_proto'

## Load Pretrained Model

In [3]:
# Pretrained model info
model_name = "op_multinet_pretraining_v2_unfrozen_standard_loss"
model_folder = os.path.join(PRETRAINED_ROOT, model_name)
loaded_iter = 3501
pretrained_model_path = os.path.join(model_folder, '%06d.pth' % loaded_iter)
train_config = utils.getDictFromYamlFilename(os.path.join(model_folder, 'training.yaml'))
pretrained_object_ids = list(np.load(os.path.join(model_folder, "object_ids.npy")))
print("Pretrained objects: ", pretrained_object_ids)

# NOTE: overwrite old log dir folder
train_config['training']['logging_dir_name'] = "FINETUNING_" + model_name

NetworkClass = OPMultiDenseCorrespondenceNetworkV2
assert train_config['dense_correspondence_network']['model_class'] == str(NetworkClass)
print(json.dumps(train_config, indent=2))

('Pretrained objects: ', ['caterpillar', 'dramamine', 'drill', 'gopro_box', 'headphones'])
{
  "device": "cuda:1", 
  "training": {
    "data_type_probabilities": {
      "DIFFERENT_OBJECT": 0, 
      "SINGLE_OBJECT_WITHIN_SCENE": 1, 
      "MULTI_OBJECT": 0, 
      "SYNTHETIC_MULTI_OBJECT": 0, 
      "SINGLE_OBJECT_ACROSS_SCENE": 0
    }, 
    "num_test_imgs_per_scene": 1, 
    "save_rate": 1000, 
    "num_workers": 5, 
    "logging_dir": "/home/ashek/code/data/pdc/trained_models/tutorials", 
    "compute_test_loss": true, 
    "sample_matches_only_off_mask": true, 
    "fraction_masked_non_matches": 0.5, 
    "batch_size": 1, 
    "domain_randomize": true, 
    "num_matching_attempts": 10000, 
    "weight_decay": 0.0001, 
    "garbage_collect_rate": 1, 
    "compute_test_loss_rate": 500, 
    "num_non_matches_per_match": 150, 
    "num_iterations": 3500, 
    "steps_between_learning_rate_decay": 250, 
    "cross_scene_num_samples": 10000, 
    "logging_dir_name": "FINETUNING_op_multi

## Object Pursuit

This should take about ~12-15 minutes with a GTX 1080 Ti

In [4]:
# dataset.get_pose_data(dataset.get_scene_list()[0])
data_config

{'logs_root_path': '/home/ashek/code/data/pdc/logs_proto',
 'multi_object_scenes_config_files': [],
 'single_object_scenes_config_files': ['10_drill_scenes.yaml',
  'caterpillar_upright.yaml',
  'dramamine.yaml',
  'gopro_box.yaml',
  'headphones.yaml',
  'mugs_brown.yaml',
  'mugs_best.yaml',
  'shoe_brown_boot.yaml',
  'mugs_red.yaml',
  'shoe_gray_nike.yaml',
  'shoe_green_nike.yaml',
  'shoe_red_nike.yaml',
  'mugs_dreamy.yaml']}

In [5]:
z_idx = 4
print("Using pretrained basis: %s" % pretrained_object_ids[z_idx])

Using pretrained basis: headphones


In [6]:
train = OPDenseCorrespondenceTraining(NetworkClass=NetworkClass, config=train_config)
train.load_dataset_from_config(config=data_config)
train.run(pretrained_model_path=pretrained_model_path, z_idx=z_idx)
print "finished training descriptor of dimension %d" %(d)

# Each batch, select a random image from a random scene
# for that image, find another image from the same scene
# but a taken from a diff view.
# then generate both pixel matches and non-matches 
# yes there is a heuristic approach, but it seeems complex
# and thus runtime is long. Prune non-matches based on:
# 1. depth is 0 (thus invalid, cannot use)
# 2. Project pixels from im1 to onto im2 and check if outside FOV
# 3. check if pixel is occluded using some method
# complicated... don't touch

Using SpartanDataset:
   - in train mode
   - number of scenes 64
   - total images:     21859
DATASET MODE: train
Using SpartanDataset:
   - in test mode
   - number of scenes 27
   - total images:     9753
DATASET MODE: test
using SINGLE_OBJECT_WITHIN_SCENE
using SINGLE_OBJECT_WITHIN_SCENE
using SINGLE_OBJECT_WITHIN_SCENE
using SINGLE_OBJECT_WITHIN_SCENE
('dir_name: ', 'FINETUNING_op_multinet_pretraining_v2_unfrozen_standard_loss')
('logging_dir:', '/home/ashek/code/data/pdc/trained_models/tutorials/FINETUNING_op_multinet_pretraining_v2_unfrozen_standard_loss')
ResNeet34 OP freeze backbone: False
Starting pursuing:
            z_dim:                            100
            output dir:                       /home/ashek/code/data/pdc/trained_models/tutorials/FINETUNING_op_multinet_pretraining_v2_unfrozen_standard_loss
            device:                           cuda:1
            pretrained_path:                  /home/ashek/code/data/pdc/trained_models/tutorials/op_multinet_pretr

/home/ashek/code/modules/dense_correspondence_manipulation/utils/utils.py:266: RuntimeWarning: invalid value encountered in arccos
  theta = 2*np.arccos(2 * np.dot(q,r)**2 - 1)



=============================start new object==============================
Starting new object:
                round:               1
                current base num:    1
                object data dir:     caterpillar
                object index:        1
                output obj dir:      /home/ashek/code/data/pdc/trained_models/tutorials/FINETUNING_op_multinet_pretraining_v2_unfrozen_standard_loss/explored_objects/obj_1
            
Current object is novel, min loss: 10000000000.0 vs thresh 0.13, most similiar object: None, start object pursuit
Z-acc pairs: []
start coefficient pursuit (first check):
coeff pursuit result dir: /home/ashek/code/data/pdc/trained_models/tutorials/FINETUNING_op_multinet_pretraining_v2_unfrozen_standard_loss/explored_objects/obj_1/coeff_pursuit
Coeffnet: found 1 Base files
Starting training:
            net type:        {net_type}
            Max epochs:      {max_epochs}
            Batch size:      {batch_size}
            Learning rate:   {lr}

Epoch 1/80:   0%|          | 0/64 [00:00<?, ?img/s]/home/ashek/.local/lib/python2.7/site-packages/torch/nn/functional.py:2589: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.")
Epoch 1/80:   2%|▏         | 1/64 [00:05<06:09,  5.87s/img]


Start epoch 1


...

Start epoch 4


Epoch 5/80:   2%|▏         | 1/64 [00:03<03:37,  3.46s/img]


Start epoch 5


Epoch 6/80:   0%|          | 0/64 [00:03<?, ?img/s]


KeyboardInterrupt: 

In [ ]:
pretrained_object_ids